In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Feature extraction from skin images

In [ ]:
from os import listdir
from PIL import Image as PImage
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
# from keras.layers.advanced_activations import LeakyReLU, ELU
from keras.models import Sequential,Model

import matplotlib.pyplot as plt

from keras.preprocessing.image import load_img

from sklearn.model_selection import StratifiedKFold

from keras_preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import img_to_array

from numpy import expand_dims

from matplotlib import pyplot

#ISIC2018 imageISIC2018
path = "/content/drive/My Drive/app/skin dataset/ISIC2018_Task3_Training_Input/"

#GroundTruth

labeldf = pd.read_csv("/content/drive/My Drive/app/skin dataset/ISIC2018_Task3_Training_GroundTruth.csv")


imagesList = listdir(path)


images = []

labels=[]


counter=1

label1counter=0

for image in imagesList:


    rowimg = load_img(path+image,target_size=(256,256))


    data= img_to_array(rowimg)



    key=image.split(".")[0]


    cls = np.where(labeldf.query('image==@key').values[0, 1:])[0]

    clslabel=cls[0]

    images.append(data.astype('uint8'))

    labels.append(clslabel)

    counter=counter+1


images = np.array(images,dtype = 'float16')

labels = np.array(labels,dtype = 'float16')



Feature extraction from skin images and data augmentation for balancing classes using Keras ImageDataGenerator

In [ ]:
from os import listdir
from PIL import Image as PImage
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
# from keras.layers.advanced_activations import LeakyReLU, ELU
from keras.models import Sequential,Model

from keras.layers.normalization import BatchNormalization

from keras.utils import to_categorical

import matplotlib.pyplot as plt

from keras.preprocessing.image import load_img

from sklearn.model_selection import StratifiedKFold

from keras_preprocessing.image import ImageDataGenerator

from keras.preprocessing.image import img_to_array

from numpy import expand_dims

from matplotlib import pyplot


path = "/content/drive/My Drive/app/skin dataset/ISIC2018_Task3_Training_Input/"

labeldf = pd.read_csv("/content/drive/My Drive/app/skin dataset/ISIC2018_Task3_Training_GroundTruth.csv")


imagesList = listdir(path)


images = []

labels=[]

f0=0
f1=0
f2=0
f3=0
f4=0
f5=0
f6=0



datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        #brightness_range=(0.9,1.1),
        fill_mode='nearest')


counter=1

label1counter=0

for image in imagesList:


    rowimg = load_img(path+image,target_size=(128,128))

    data= img_to_array(rowimg)


    key=image.split(".")[0]


    cls = np.where(labeldf.query('image==@key').values[0, 1:])[0]



    clslabel=cls[0]



    images.append(data.astype('uint8'))

    labels.append(clslabel)



    lenght=0



    # MEL / 0 ==>1113
    # NV  / 1 ==>6705
    # BCC  / 2 ==>514
    # AKIES  / 3 ==>327
    # BKL  / 4  ==>1099
    # DF  / 5 ==>115
    # VASC  / 6 ==>142

    if clslabel==1 :

       f1=f1+1
       lenght=0

    if clslabel==0 :

       lenght=5

       f0=f0+1+0


    if clslabel==4 :

       lenght=5

       f4=f4+1+0

    if clslabel==2 :

       lenght=19

       f2=f2+1+19

    if clslabel==3 :

       lenght=17

       f3=f3+1+8


    if clslabel==6 :

       lenght=41

       f6=f6+1+9


    if clslabel==5 :

       lenght=42

       f5=f5+1+25


    for i in range(lenght):


        samples = expand_dims(data, 0)


        it = datagen.flow(samples , batch_size=1)


        batch =it.next()


        img2 =batch[0].astype('uint8')



        images.append(img2)


        labels.append(clslabel)


    counter=counter+1


images = np.array(images,dtype = 'float16')

labels = np.array(labels,dtype = 'float16')



Create pre-trained models dynamically and calculate fitness

In [ ]:

import random

import numpy

from tensorflow.keras.optimizers import Adadelta,Adagrad,SGD,RMSprop,Adam,SGD

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception

from tensorflow.python.keras.applications.nasnet import NASNetLarge
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.layers import SeparableConv2D,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import Callback
import gc


#0 Adam
#1 SGD
#2 RMSprop
#3 Adadelta
#4 Adagrad

#Hyperparameter names and ranges

num_optimizerlist = [0,1,2,3,4]

num_learningratelist  = [0.1,0.01,0.001,0.0001,0.00001]

num_denselayer_neroun  = [64,128,256,512,1024,2048,4096]

num_droprate_list  = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

num_batchsizelist = [32,32,32]

#VGG16 =>19
#InceptionV3 =>311 1 top
#Resnet-50 =>175 1
#InceptionRes =>780 1

#number of layers for pre-trained models()
LayerNumber=175

def CreateBaseModel(modelName):

  if modelName =="VGG16":

    base_model = VGG16(weights='imagenet',include_top= False,input_shape=(256,256,3))

    print("model is vgg16")

  if modelName =="InceptionV3":

    base_model = InceptionV3(weights='imagenet',include_top= False,input_shape=(256,256,3))

    print("model is InceptionV3")

  if modelName =="ResNet-50":

    base_model = ResNet50(weights='imagenet',include_top= False,input_shape=(256,256,3))

    print("model is ResNet-50")


  if modelName =="InceptionResNetV2":

    base_model = InceptionResNetV2(weights='imagenet',include_top= False,input_shape=(256,256,3))

    print("model is InceptionResNetV2")


  if modelName =="DenseNet201":

    base_model = DenseNet201(weights='imagenet',include_top= False,input_shape=(256,256,3))

    print("model is DenseNet201")


  if modelName =="NASNetLarge":

    base_model = NASNetLarge(weights='imagenet',include_top= False,input_shape=(331,331,3))

    print("model is NASNetLarge")


  if modelName =="Xception":

    base_model = Xception(weights='imagenet',include_top= False,input_shape=(256,256,3))

    print("model is Xception")

  return base_model


def createDeeepModel(optimizercode,learningrate,denselayer,droprate,batchsize,freezlayer):

  print('optimizercode : ',optimizercode,'learningrate ; ',learningrate,'denselayer : ',denselayer,'droprate : ',droprate,'freezlayer : ',freezlayer)
  print('*******************************************************')
  base_model= CreateBaseModel("ResNet-50")

  freezlayer=int(freezlayer)


  batchsize=int (batchsize)

  denselayer=int(denselayer)

  optimizercode=int(optimizercode)

  if optimizercode ==0 :

    optimizer=Adam(learning_rate=learningrate)

  if optimizercode ==1 :

    optimizer=SGD(learning_rate=learningrate)

  if optimizercode ==2 :

    optimizer=RMSprop(learning_rate=learningrate)

  if optimizercode ==3 :

    optimizer=Adadelta(learning_rate=learningrate)

  if optimizercode ==4 :

    optimizer=Adagrad(learning_rate=learningrate)

  x = base_model.output

  x = GlobalAveragePooling2D()(x)

  x = Dense(denselayer, activation='relu')(x)

  x = Dropout(droprate)(x)

  predictions = Dense(7, activation= 'softmax')(x)

  model = Model(base_model.input,predictions)

  for layer in model.layers[:freezlayer]:

    layer.trainable = False

  for layer in model.layers[freezlayer:]:

    layer.trainable = True

  model.compile(loss='SparseCategoricalCrossentropy', optimizer=optimizer, metrics=['accuracy'])


  return(model)


population=[]

class MyCustomCallback(Callback):


  def on_epoch_end(self, epoch, logs=None):


    gc.collect()


#Fitness function

def Fitness(optimizer,learningrate,denselayer,droprate,batchsize,freezlayer):

    modelname=''

    model=createDeeepModel(optimizer,learningrate,denselayer,droprate,batchsize,freezlayer)

    callbacks = [EarlyStopping(monitor='val_accuracy', patience=3),MyCustomCallback()]

    batch_size=int(batchsize)

    model.fit(trainimages, trainlabels, epochs=30, batch_size=32,callbacks=callbacks,validation_data=(validationimages,validationlabels), verbose=1)

    scores= model.evaluate(testimages, testlabels)

    error=1-scores[1]

    del model

    gc.collect()

    return error


In [ ]:
pip install scikit-opt

In [ ]:
from sko.DE import DE

import time

start_time = time.time()

de = DE(func=Fitness, n_dim=6, size_pop=6, max_iter=10, lb = [0,0.00001,64,0.0,32,0], ub = [4,0.1,4096,0.9,128,175])

best_x, best_y = de.run()

print('best_x:', best_x, '\n', 'best_y:', best_y)

print("--- %s seconds ---" % (time.time() - start_time))

import matplotlib.pyplot as plt

plt . plot ( de . gbest_y_hist )
plt . show ()

print('best_x:', best_x, '\n', 'best_y:', best_y)
# %% Plot the result
import pandas as pd
import matplotlib.pyplot as plt

Y_history = pd.DataFrame(de.all_history_Y)
fig, ax = plt.subplots(2, 1)
ax[0].plot(Y_history.index, Y_history.values, '.', color='red')
Y_history.min(axis=1).cummin().plot(kind='line')
plt.show()

In [ ]:
from sko.GA import GA

import time

start_time = time.time()

ga  = GA(func=Fitness, n_dim=6, size_pop=5, max_iter=10, lb = [0,0.00001,64,0.0,32,0], ub = [4,0.1,4096,0.9,128,71])

best_x, best_y = ga.run()
print('best_x:', best_x, '\n', 'best_y:', best_y)

print("--- %s seconds ---" % (time.time() - start_time))

import matplotlib.pyplot as plt

plt . plot ( ga . gbest_y_hist )
plt . show ()

In [ ]:
from sko.PSO import PSO
import time

start_time = time.time()

pso  =  PSO ( func = Fitness , n_dim = 6 , pop = 5 , max_iter = 10 , lb = [0,0.00001,64,0.0,32,0], ub = [4,0.1,4096,0.9,128,175], w = 0.8 , c1 = 0.5 , c2 = 0.5 )

pso.run()

print('best_x is ', pso.gbest_x, 'best_y is', pso.gbest_y)

print("--- %s seconds ---" % (time.time() - start_time))

import matplotlib.pyplot as plt

plt . plot ( pso . gbest_y_hist )
plt . show ()

Train Pretrained models using optimal hyperparameters
َ &msave model to drive

In [ ]:

from tensorflow.keras.optimizers import Adadelta,Adagrad,SGD,RMSprop,Adam,SGD
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.layers import SeparableConv2D,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import Callback
import gc



class MyCustomCallback(Callback):


  def on_epoch_end(self, epoch, logs=None):


    gc.collect()

#0 Adam
#1 SGD
#2 RMSprop
#3 Adadelta
#4 Adagrad

learningrate=0.0823

optimizer=Adadelta(learning_rate=learningrate)


droprate= 0.6

denselayer=2051

freezlayer=40

base_model = ResNet50(weights='imagenet',include_top= False,input_shape=(256,256,3))

x = base_model.output

x = GlobalAveragePooling2D()(x)

x = Dense(denselayer, activation='relu')(x)

x = Dropout(droprate)(x)

predictions = Dense(7, activation= 'softmax')(x)

model = Model(base_model.input,predictions)

for layer in model.layers[:freezlayer]:

 layer.trainable = False

for layer in model.layers[freezlayer:]:

 layer.trainable = True


model.compile(loss='SparseCategoricalCrossentropy',optimizer=optimizer, metrics=['accuracy'])


model.fit(trainimages, trainlabels, epochs=30, batch_size=32,validation_data=(validationimages,validationlabels), verbose=1)

scores= model.evaluate(testimages, testlabels)

prediction= model.predict(testimages)

print(scores[1])

#del model

#gc.collect()

error=1-scores[1]
model.save()


Deef featue extraction and concatenaton

In [ ]:
from tensorflow.keras.models import load_model

from tensorflow.keras.models import Sequential,Model


densemodel = load_model('/content/drive/My Drive/app/dense.h5')


dense_pre_model = Model(densemodel.inputs, densemodel.layers[-2].output) # Dense(128,...)



#incepmodel = load_model('/content/drive/My Drive/app/inception.h5')


#incep_pre_model = Model(incepmodel.inputs, incepmodel.layers[-2].output) # Dense(128,...)



resmodel = load_model('/content/drive/My Drive/app/resnet.h5')


res_pre_model = Model(resmodel.inputs, resmodel.layers[-2].output) # Dense(128,...)


print("Extrcting res Features ...")

restrainfeatures = res_pre_model.predict(trainimages, batch_size=16)

restestfeatures = res_pre_model.predict(testimages, batch_size=16)


#print("Extrcting incep train Features ...")

#inceptrainfeatures = incep_pre_model.predict(trainimages, batch_size=32)

#inceptestfeatures = incep_pre_model.predict(testimages, batch_size=32)


print("Extrcting incep train Features ...")

densetrainfeatures = dense_pre_model.predict(trainimages, batch_size=32)

densetestfeatures = dense_pre_model.predict(testimages, batch_size=32)



print(restrainfeatures.shape)
#print(inceptrainfeatures.shape)
print(densetestfeatures.shape)

newtrain=np.hstack((densetrainfeatures,restrainfeatures))
newtrain = np.array(newtrain,dtype = 'float16')

newtest=np.hstack((densetestfeatures,restestfeatures))
newtest = np.array(newtest,dtype = 'float16')

print(newtrain.shape)
print(newtest.shape)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=20)
classifier.fit(featureselectedtrain, trainlabels)


y_pred = classifier.predict(featureselectedtest)

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
print(confusion_matrix(testlabels,y_pred))
print(classification_report(testlabels,y_pred,digits=4))
print(accuracy_score(testlabels,y_pred))
